In [1]:
!pip install langgraph langchain langchain-core langchain-classic transformers accelerate sentencepiece bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 3.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 34.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:

In [2]:
!pip install langchain langchain-chroma langchain-huggingface langchain-community rank_bm25 flashrank --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 102.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 100.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [3]:
import torch
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from flashrank import Ranker, RerankRequest  # <--- CRITICAL IMPORT
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline
from langchain_classic.storage import LocalFileStore, EncoderBackedStore
from langchain_classic.retrievers import ParentDocumentRetriever, EnsembleRetriever, ContextualCompressionRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_classic.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_classic.retrievers.document_compressors import FlashrankRerank
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.retrievers import BaseRetriever
from langchain_core.stores import BaseStore
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from typing import List, Dict, Any, Optional

2025-11-25 12:26:52.439746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764073612.641308      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764073612.691229      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
model_id = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True,
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.1,
    top_p=0.95,
    repetition_penalty=1.1,
    do_sample=True
)

In [ ]:
# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=pipe)

print("✓ Qwen model loaded successfully!")

In [5]:
def chat(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [6]:
print(chat("Explain what a transformer model is in simple terms."))

Explain what a transformer model is in simple terms. A transformer model is a type of artificial intelligence model used for processing sequential data, such as text. It was introduced in 2017 by a team at Google's research lab. Unlike earlier models like RNNs (recurrent neural networks) which process data sequentially, transformers use a mechanism called self-attention to process all parts of the input simultaneously. This allows them to better capture relationships between words in a sentence and make them more efficient at handling long-range dependencies. Transformers have been widely adopted in natural language processing tasks such as machine translation, text generation, and question answering. They are also the foundation for many large language models like BERT, GPT, and T5. The key components of a transformer model include the encoder and decoder layers, which consist of self-attention mechanisms and feed-forward neural networks. These models are trained using large amounts o

# Building Retriever

In [4]:
VECTOR_DB_PATH = "/kaggle/input/ncert-study-buddy-stores/chroma_db"
DOC_STORE_PATH = "/kaggle/input/ncert-study-buddy-stores/doc_store"

In [5]:
import os
import shutil

writable_path = '/kaggle/working/chroma_db'
read_only_path = VECTOR_DB_PATH

if not os.path.exists(writable_path):
    print(f"Copying database from {read_only_path} to {writable_path}...")
    shutil.copytree(read_only_path, writable_path)
    print("Copy complete.")
else:
    print("Database already exists in working directory.")

Copying database from /kaggle/input/ncert-study-buddy-stores/chroma_db to /kaggle/working/chroma_db...
Copy complete.


In [6]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

parent_store = EncoderBackedStore(
    store=LocalFileStore(DOC_STORE_PATH),
    key_encoder=lambda x: str(x),
    value_deserializer=pickle.loads,
    value_serializer=pickle.dumps
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
vectorstore = Chroma(
    collection_name="study_chunks",
    embedding_function=embeddings,
    persist_directory=writable_path # Use disk persistence
)

In [8]:
pdr_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=parent_store,
    child_splitter=RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0), # Disable auto-splitting
    parent_splitter=None # Disable auto-splitting
)

pdr_retriever.search_kwargs = {"k": 20}

In [9]:
class BM25ParentRetriever(BaseRetriever):
    """
    Searches Child chunks via BM25, but returns the associated Parent Documents.
    Ensures output consistency with ParentDocumentRetriever.
    """
    bm25_retriever: BM25Retriever
    docstore: BaseStore
    id_key: str = "doc_id"

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        # 1. Run Keyword Search on granular children
        child_docs = self.bm25_retriever.invoke(query)
        
        # 2. Extract unique Parent IDs
        parent_ids = []
        for doc in child_docs:
            if self.id_key in doc.metadata:
                p_id = doc.metadata[self.id_key]
                if p_id not in parent_ids:
                    parent_ids.append(p_id)
        
        # 3. Fetch full Parents from the shared store
        # mget returns a list of (Document or None)
        parent_docs = self.docstore.mget(parent_ids)
        
        # 4. Filter out Nones (in case a parent was deleted/missing)
        return [d for d in parent_docs if d is not None]

In [10]:
import json

with open('/kaggle/input/ncert-study-buddy-chunks/paragraphs.json', 'r', encoding='utf-8') as file:
    children_data = json.load(file)

child_docs_to_store = []
for c_data in children_data:
    doc = Document(
        page_content=c_data["content"],
        metadata=c_data["metadata"]
    )
    # CRITICAL: Link Child -> Parent
    # The PDR looks for the key 'doc_id' in metadata to find the parent
    doc.metadata["doc_id"] = c_data["parent_id"]
    child_docs_to_store.append(doc)

In [11]:
base_bm25 = BM25Retriever.from_documents(child_docs_to_store)
base_bm25.k = 20

In [12]:
bm25_parent_retriever = BM25ParentRetriever(
    bm25_retriever=base_bm25,
    docstore=parent_store
)

In [13]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[pdr_retriever, bm25_parent_retriever],
    weights=[0.5, 0.5]
)

In [14]:
reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
compressor = CrossEncoderReranker(model=reranker_model, top_n=5)
# compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [15]:
final_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=ensemble_retriever
)

In [18]:
print("Test 1: General Search")
print("=" * 80)
result1 = final_retriever.invoke("What is Battle of Plassey?")
print(result1)
print("\n")

Test 1: General Search
[Document(metadata={'pages': '10 - 15', 'chapter': 2, 'book': 'History 8'}, page_content='Mercantile – A business enterprise that makes profit primarily through trade, buying goods cheap and selling them at higher prices \nEast India Company Comes East \n\n In 1800, the East India Company acquired a charter from the ruler of England, Queen Elizabeth I, granting it the sole right to trade with the East. This meant that no other trading group in England could compete with the East India Company. With this charter the Company could venture across the oceans, looking for new lands from which it could buy goods at a cheap price, and carry them back to Europe to sell at higher prices. The Company did not have to fear competition from other English trading companies. Mercantile trading companies in those days made profit primarily by excluding competition, so that they could buy cheap and sell dear. \nThe royal charter, however, could not prevent other European powers f